In [3]:
import pickle
import pandas as pd
import numpy as np
from IPython.display import Audio
import torch
import torchaudio
from IPython.display import Audio, display
import json
import os

In [4]:
def df_for_snr(snr_path):
    stats_path = os.path.join(snr_path, "stats_dns.json")
    with open(stats_path) as f:
        data = json.load(f)
    df = pd.DataFrame.from_dict(data).T.reset_index()
    for i in range(len(df)):
        df.at[i, "index"] = df["index"][i].split("s")[1]
    df["index"] = df["index"].astype(float)
    df = df.sort_values(by=['index']).reset_index(drop=True)
    df= df.rename(columns={"index": "s"})
    df.to_excel(os.path.join(snr_path,"measurements.xlsx"))

    return df
    

In [38]:
from pesq import pesq
from pystoi import stoi
clean = "/data/ephraim/datasets/known_noise/undiff/exp_ar_h_0.99/c/clean_wav/LJ045-0072.wav"

snr5path = "/data/ephraim/datasets/known_noise/undiff/exp_ar_h/c/noisy_wav/digits_snr5_power0.001966120132704692_var0.0003768711770958883.wav"
snr10path = "/data/ephraim/datasets/known_noise/undiff/exp_ar_h/c/noisy_wav/digits_snr10_power0.0006217417772859335_var0.00011917713040916882.wav"
snr15path = "/data/ephraim/datasets/known_noise/undiff/exp_ar_h/c/noisy_wav/digits_snr15_power0.00019661201327046915_var3.768711770958882e-05.wav"
# enhancedpath = "/data/ephraim/datasets/known_noise/undiff/exp_ar_h/a/enhanced_constant/snr15/s0.05/digits_snr15_power0.0003041422736918628_var5.7854771367023836e-05.wav"

WAVEFORM_SPEECH, SAMPLE_RATE_SPEECH = torchaudio.load(clean)
WAVEFORM_NOISE, SAMPLE_RATE_NOISE = torchaudio.load(snr15path)
print(pesq(16000,WAVEFORM_SPEECH[0].numpy(),
                    WAVEFORM_NOISE[0].numpy(),
                    mode="wb",))
stoi(WAVEFORM_SPEECH[0].numpy(),
                    WAVEFORM_NOISE[0].numpy(),
                    16000,
                    extended=False,)

1.340324878692627


0.9084301417686218

In [3]:

import torch
import librosa
from asteroid.metrics import get_metrics

In [50]:


# wav_file_path = 'path/to/your/file.wav'

# # Load the .wav file
# audio, sample_rate = librosa.load(wav_file_path, sr=None)  # sr=None to preserve original sample rate

# # Convert audio to tensor
# audio_tensor = torch.tensor(audio).unsqueeze(0)  # Adding batch dimension

# # Assuming you have target and estimate tensors (here we use the same for demonstration)
# target_tensor = audio_tensor  # For demonstration purposes
# estimate_tensor = audio_tensor  # For demonstration purposes

# Compute metrics
metrics = get_metrics(clean=WAVEFORM_SPEECH[0].numpy(), mix=WAVEFORM_NOISE[0].numpy(),estimate=WAVEFORM_NOISE[0].numpy(), sample_rate=16000)

# Print metrics
print(metrics)

{'input_si_sdr': 13.555511474609375, 'input_sdr': 13.581198577406244, 'input_sir': inf, 'input_sar': 13.581198577406244, 'input_stoi': 0.8948478023683961, 'input_pesq': 1.3012539148330688, 'si_sdr': 13.555511474609375, 'sdr': 13.581198577406244, 'sir': inf, 'sar': 13.581198577406244, 'stoi': 0.8948478023683961, 'pesq': 1.3012539148330688}


In [51]:
stoi(WAVEFORM_SPEECH[0].numpy(),
                    WAVEFORM_NOISE[0].numpy(),
                    16000,
                    extended=False,
                )

0.8948478023683961

In [39]:
enhance_dir = os.path.join(rootconstant, "snr15/s"+str(0.01))
pkl_results_file = os.path.join(enhance_dir, "stats.pickle")
print(pkl_results_file)
if os.path.exists(pkl_results_file):
    print("b")
    with open(pkl_results_file, "rb") as handle:
        df5 = pd.read_pickle(handle)
        stats5 = df5.describe()
        print(stats5)

/data/ephraim/datasets/known_noise/undiff/exp_ar_h/a/enhanced_constant/snr15/s0.01/stats.pickle
b
       pesq_noisy  stoi_noisy  pesq_enhanced  stoi_enhanced  pesq_diff  \
count    3.000000    3.000000       3.000000       3.000000   3.000000   
mean     1.145743    0.820832       1.107925       0.681727  -0.037818   
std      0.138470    0.080827       0.000000       0.000000   0.138470   
min      1.035795    0.734588       1.107925       0.681727  -0.193329   
25%      1.067988    0.783824       1.107925       0.681727  -0.092792   
50%      1.100181    0.833060       1.107925       0.681727   0.007745   
75%      1.200717    0.863954       1.107925       0.681727   0.039937   
max      1.301254    0.894848       1.107925       0.681727   0.072130   

       stoi_diff  
count   3.000000  
mean   -0.139105  
std     0.080827  
min    -0.213121  
25%    -0.182227  
50%    -0.151333  
75%    -0.102097  
max    -0.052861  


In [8]:
from pathlib import Path
# root_ = Path("/data/ephraim/datasets/known_noise/undiff/exp_ar_h/c/")
# root_ = Path("/data/ephraim/datasets/known_noise/undiff/exp_ar_h_0.99/c")
root_ = Path("/data/ephraim/datasets/known_noise/undiff/exp_ar_h_white/a")

root40 = root_ / "enhanced_40/"
root50 =  root_ / "enhanced_50/"
root60 =  root_ / "enhanced_60/"
root90 =  root_ / "enhanced_90/"

rootlinear =  root_ / "enhanced_linear/"
rootconstant = root_ / "enhanced_constant/"
rootsinusoidal =  root_ / "enhanced_sinusoidal/"


In [9]:
df = df_for_snr(os.path.join(root60, "snr5"))
print(max(df.pesq))
print(df[df.pesq == max(df.pesq)]["stoi"])
print(df[df.pesq == max(df.pesq)]["s"])
df

1.0337204933166504
0    0.3153480839941711
Name: stoi, dtype: object
0    0.01
Name: s, dtype: float64


,s,BAK,OVRL,SIG,args,enhance_dir,pesq,pesq_diff,stoi,stoi_diff
0,0.01,3.8215554169575405,2.5321680985685333,2.8476040193132124,Namespace(enhanced_dir='/data/ephraim/datasets...,/data/ephraim/datasets/known_noise/undiff/exp_...,1.0337204933166504,-0.026183009147644043,0.3153480839941711,-0.5260781178839666


In [5]:
df = df_for_snr(os.path.join(root60, "snr5"))
print(max(df.pesq))
print(df[df.pesq == max(df.pesq)]["stoi"])
print(df[df.pesq == max(df.pesq)]["s"])

df

1.3799630403518677
14    0.8530429185718731
Name: stoi, dtype: object
14    0.17
Name: s, dtype: float64


,s,pesq,stoi,pesq_diff,stoi_diff,OVRL,SIG,BAK,enhance_dir,args
0,0.01,1.0271283388137817,0.3608944366879183,-0.14383288224538168,-0.47947684132638174,1.675498456506092,2.3179688549254176,3.407133316669128,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
1,0.04,1.1168324947357178,0.6643616419847752,-0.05412872632344564,-0.1760096360295249,2.188643508990186,2.511838801770795,3.6889125885542313,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
2,0.05,1.151658535003662,0.6803924809522389,-0.0193026860555013,-0.15997879706206108,2.6136666855093997,2.980439119787928,3.7306778215473186,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
3,0.06,1.165947437286377,0.7572962436486216,-0.005013783772786458,-0.08307503436567854,2.785551262555258,3.1274824277794253,3.8601268489024996,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
4,0.07,1.1953725814819336,0.7712611711114395,0.024411360422770183,-0.06911010690286057,2.7842642479520907,3.084797194542489,3.9430881280694887,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
5,0.08,1.2148081064224243,0.7969008230249818,0.0438468853632609,-0.04347045498931845,2.684999707720435,3.0835318659291144,3.66341125736333,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
6,0.09,1.3369107246398926,0.8268875927685926,0.16594950358072916,-0.013483685245707533,2.9527887096314664,3.259763388889884,3.90914501551239,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
7,0.10,1.2420588731765747,0.8360855701128612,0.0710976521174113,-0.0042857079014387844,2.690049366476997,3.125134985623756,3.5915294699074045,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
8,0.11,1.361473560333252,0.8404649452086611,0.19051233927408853,9.366719436099125e-05,2.6918681451918203,3.0262788861361436,3.7586584227005897,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
9,0.12,1.3325084447860718,0.8457943819757304,0.16154722372690836,0.005423103961430309,2.5541156461147545,2.9202046815433906,3.6272442842427886,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...


In [12]:
df = df_for_snr(os.path.join(rootconstant, "snr5"))
print(max(df.pesq))
df

1.1946148872375488


,s,pesq,stoi,pesq_diff,stoi_diff,OVRL,SIG,BAK,enhance_dir,args
0,0.01,1.022957444190979,0.4198762016825633,-0.1227857271830241,-0.40095557737259346,2.643365509901421,3.0256898923046665,3.755884478472936,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
1,0.04,1.1472238302230835,0.803464672797916,0.0014806588490804036,-0.017367106257240843,2.2467800941186487,3.3010025287889015,2.4209744771975723,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
2,0.05,1.1801098585128784,0.8638902670308247,0.03436668713887533,0.04305848797566788,2.391246599585339,3.398785482117778,2.5560648175239074,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
3,0.06,1.1946148872375488,0.8630571288458259,0.04887171586354574,0.04222534979066902,2.5513869795429542,3.4517166656255673,2.769356453884298,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
4,0.07,1.1643410921096802,0.865348725527753,0.018597920735677082,0.04451694647259615,2.4486718461027768,3.5365179837747363,2.4501732748117617,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
5,0.08,1.1513864994049072,0.8621343946718781,0.005643328030904134,0.041302615616721404,2.3086791970469673,3.5090764658667455,2.2494320479995213,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
6,0.09,1.1368775367736816,0.8505936123655014,-0.008865634600321451,0.029761833310344505,2.307815813559109,3.534660844894098,2.2452589423673137,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
7,0.10,1.1166263818740845,0.8473445486089638,-0.02911678949991862,0.026512769553807065,2.3115127991286464,3.55947137397122,2.2197132345164152,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
8,0.11,1.1078381538391113,0.839714950768187,-0.03790501753489176,0.018883171713030073,2.2059519497228175,3.5474887158970145,2.011982563565733,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
9,0.12,1.102105975151062,0.829838428116671,-0.04363719622294108,0.009006649061514013,2.1070390132657897,3.507732206258418,1.8815939849421546,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...


In [10]:
df = df_for_snr(os.path.join(root60, "snr5"))
print(max(df.pesq))
df

1.3547475337982178


,s,pesq,stoi,pesq_diff,stoi_diff,OVRL,SIG,BAK,enhance_dir,args
0,0.01,1.0280380249023438,0.2205348748343944,-0.11770514647165935,-0.6002969042207624,2.5821971983023673,2.8847652149629885,3.827388877428699,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
1,0.04,1.1173017024993896,0.6988403514119699,-0.028441468874613445,-0.12199142764318698,2.661393988559077,2.982303400579439,3.8090712771251054,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
2,0.05,1.1538439989089966,0.7292166750996262,0.00810082753499349,-0.09161510395553067,2.5860106239324705,2.918025257924709,3.7859156035551744,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
3,0.06,1.2630536556243896,0.7836766511923149,0.11731048425038655,-0.037155127862841954,2.826515434708167,3.1449004257268567,3.9001891847446575,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
4,0.07,1.2653493881225586,0.8188582746304759,0.1196062167485555,-0.001973504424680842,3.0963938569404235,3.361930018112385,4.052067428531795,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
5,0.08,1.30306875705719,0.8113574721726385,0.15732558568318686,-0.009474306882518194,2.9747119756328386,3.261974347365123,3.9862500110000063,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
6,0.09,1.2854890823364258,0.8295548705701427,0.1397459109624227,0.008723091514985922,2.839866500023882,3.1429922274287825,3.909891179147993,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
7,0.10,1.3240017890930176,0.8293700777769532,0.1782586177190145,0.008538298721796308,2.854918652904417,3.148753138637402,3.9508708442614924,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
8,0.11,1.2958664894104004,0.8407890775130992,0.1501233180363973,0.019957298457942325,2.880727236300278,3.2070950699678606,3.8978137910227417,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
9,0.12,1.30734121799469,0.8452712397025506,0.16159804662068686,0.024439460647393735,3.126131651699687,3.4553539575436822,3.963394599206964,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...


In [8]:
df = df_for_snr(os.path.join(root50, "snr5"))
print(max(df.pesq))
df

1.3447929620742798


,s,pesq,stoi,pesq_diff,stoi_diff,OVRL,SIG,BAK,enhance_dir,args
0,0.01,1.0707439184188843,0.21717561428342905,-0.07499925295511882,-0.6036561647717277,2.692440007941762,2.99547263425356,3.870030156232887,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
1,0.04,1.0946760177612305,0.683014135720463,-0.05106715361277262,-0.13781764333469385,2.628197573474648,2.9464464524264526,3.836497543446048,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
2,0.05,1.1789579391479492,0.7305411897086568,0.03321476777394613,-0.09029058934650007,2.7216338514478626,3.0042645687354494,3.918007193562509,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
3,0.06,1.2161773443222046,0.7736044044721749,0.07043417294820149,-0.04722737458298202,2.6680965074680154,2.964591004353301,3.8969516064521867,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
4,0.07,1.232875108718872,0.7795850380183401,0.08713193734486897,-0.041246741036816616,2.9749840114139707,3.2613964675393055,3.983048123179443,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
5,0.08,1.2742427587509155,0.8029074477016688,0.12849958737691244,-0.01792433135348807,2.838107182286265,3.147608557368156,3.9061221284272145,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
6,0.09,1.3266525268554688,0.8178245503085373,0.18090935548146567,-0.0030072287466195538,3.0371068807153,3.3196432291544697,4.016039552378761,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
7,0.10,1.2816250324249268,0.8279769055799976,0.13588186105092367,0.007145126524840746,2.943743922527458,3.209682073265111,3.987823499244501,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
8,0.11,1.3424872159957886,0.839271683785045,0.1967440446217855,0.018439904729888063,3.0526913810529335,3.3182535685531107,4.038926220413257,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
9,0.12,1.2858961820602417,0.8341192785544923,0.14015301068623862,0.013287499499335572,3.044873485162729,3.3522937274962494,3.9731403308375275,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...


In [9]:
df = df_for_snr(os.path.join(root40, "snr5"))
print(max(df.pesq[1:]))
df

1.354715347290039


,s,pesq,stoi,pesq_diff,stoi_diff,OVRL,SIG,BAK,enhance_dir,args
0,0.01,3.199397087097168,0.23327341329614415,2.053653915723165,-0.5875583657590128,2.537409178742718,2.874587533446576,3.903794391970983,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
1,0.04,1.0972042083740234,0.6571884524768428,-0.04853896299997965,-0.16364332657831407,2.6912783191877203,3.0038447869574236,3.8757294731309715,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
2,0.05,1.1637510061264038,0.7151977993762135,0.018007834752400715,-0.10563397967894332,2.489424989055585,2.843681821834757,3.7145368485522687,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
3,0.06,1.1982425451278687,0.7600711122333018,0.05249937375386556,-0.060760666821855026,2.685438353725032,3.0083522924040706,3.8168751240476886,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
4,0.07,1.238625168800354,0.7528161234668233,0.0928819974263509,-0.06801565558833349,2.7599161087719213,3.0365232142289917,3.9335892550803875,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
5,0.08,1.2801117897033691,0.7730409406166029,0.13436861832936606,-0.04779083843855384,2.637821930182318,2.977011076212178,3.801853767722403,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
6,0.09,1.2837543487548828,0.7946490034443211,0.13801117738087973,-0.026182775610835745,2.7442469184481006,3.079304835133623,3.8262362199311957,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
7,0.10,1.2992762327194214,0.821391190156238,0.1535330613454183,0.0005594111010810051,2.906188424403811,3.198711426515644,3.9639656764241873,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
8,0.11,1.2942956686019897,0.8234291764171647,0.14855249722798666,0.002597397362007777,2.9081472170290295,3.1805104569416516,4.0030149302144125,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
9,0.12,1.3166111707687378,0.8231851236437565,0.1708679993947347,0.002353344588599556,2.8506592339210166,3.1352401903639406,3.9441065130075086,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...


In [20]:
df_for_snr(os.path.join(root, "snr10"))

,s,pesq,stoi,pesq_diff,stoi_diff,OVRL,SIG,BAK,enhance_dir,args
0,0.08,1.3363275527954102,0.8313439828647876,0.19058438142140707,0.010512203809630768,3.008532975097011,3.2849140069696126,4.003008946114903,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
1,0.09,1.3694021701812744,0.8440864170032581,0.22365899880727133,0.023254637948101304,3.067142700739397,3.3288875880494393,4.03347685921523,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
2,0.10,1.4475719928741455,0.8469163546661753,0.3018288215001424,0.026084575611018474,2.8402440488674108,3.1685712275310154,3.8769413575467437,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
3,0.11,1.3462779521942139,0.8512710462943627,0.20053478082021078,0.03043926723920583,2.9843682842886423,3.2506307293632823,4.021016291325195,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
4,0.12,1.4107733964920044,0.8468873037465127,0.2650302251180013,0.02605552469135582,2.9204127467389327,3.20951995772565,3.963281631383091,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...


In [22]:
df_for_snr(os.path.join(root, "snr5"))

,s,pesq,stoi,pesq_diff,stoi_diff,OVRL,SIG,BAK,enhance_dir,args
0,0.08,1.2866277694702148,0.7958022881010288,0.14088459809621176,-0.025029490954128025,2.939316116153949,3.201993595968438,4.0326263232035,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
1,0.09,1.3068293333053589,0.8149432461417239,0.1610861619313558,-0.00588853291343298,2.886819158264178,3.1726079279454544,3.987465393895875,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
2,0.10,1.3198009729385376,0.8231705318137955,0.17405780156453451,0.002338752758638781,3.0347695593135637,3.3308817152656927,3.999094025913247,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
3,0.11,1.3300397396087646,0.8125959235510477,0.18429656823476157,-0.008235855504109138,2.9444836401109664,3.257530809917793,3.9329797674713602,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
4,0.12,1.2993483543395996,0.8248268862315905,0.15360518296559653,0.003995107176433661,3.049432716520656,3.347135369913055,3.9948876421372965,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...


In [23]:
df_for_snr(os.path.join(root3, "snr15"))

,s,pesq,stoi,pesq_diff,stoi_diff,OVRL,SIG,BAK,enhance_dir,args
0,0.08,1.4534249305725098,0.8531579724764052,0.2824637095133464,0.01278669446210512,2.7086637069256647,3.0981661968068757,3.6788305656001907,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
1,0.09,1.5040901899337769,0.863698264289242,0.33312896887461346,0.023326986274942008,2.6663326175651108,3.0232076425415717,3.739285336256575,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
2,0.10,1.452805995941162,0.8610060151024171,0.2818447748819987,0.020634737088117028,2.841425097561246,3.186895680824097,3.847254098367715,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
3,0.11,1.4896039962768555,0.8700805086371362,0.3186427752176921,0.029709230622836036,2.73172976071206,3.122046405942147,3.6857826761910366,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
4,0.12,1.5052533149719238,0.876972730154538,0.33429209391276044,0.03660145214023791,2.776574656105288,3.193859475018854,3.654566616366535,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...


In [24]:
df_for_snr(os.path.join(root3, "snr10"))

,s,pesq,stoi,pesq_diff,stoi_diff,OVRL,SIG,BAK,enhance_dir,args
0,0.08,1.3116207122802734,0.8381862279224187,0.14065949122111002,-0.002185050091881302,2.9724788753275675,3.305533353898163,3.895674881816047,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
1,0.09,1.4776527881622314,0.8426161693062738,0.30669156710306805,0.002244891291973694,2.895651519960565,3.1898450191110967,3.924122279758682,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
2,0.10,1.444916009902954,0.8523864476521482,0.2739547888437907,0.0120151696378481,2.817817159229057,3.2517856899804976,3.645233583214438,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
3,0.11,1.4609041213989258,0.8588395982473656,0.2899429003397624,0.01846832023306544,2.93425350474876,3.2926312865336684,3.8152852191401116,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
4,0.12,1.5091910362243652,0.8803922800553936,0.33822981516520184,0.04002100204109349,2.8724889197199044,3.2265463533517953,3.804683208267676,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...


In [25]:
df_for_snr(os.path.join(root3, "snr5"))

,s,pesq,stoi,pesq_diff,stoi_diff,OVRL,SIG,BAK,enhance_dir,args
0,0.08,1.2062665224075317,0.8034713250469357,0.035305301348368325,-0.03689995296736426,2.7970824127653757,3.1648422602186685,3.7431165591315465,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
1,0.09,1.3068782091140747,0.8292934009566194,0.1359169880549113,-0.011077877057680707,2.8318605954805363,3.164703786655263,3.8550084965176765,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
2,0.10,1.3275423049926758,0.8454350783685906,0.15658108393351236,0.0050638003542905485,2.7990791422242083,3.1164520407672303,3.8320517280745974,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
3,0.11,1.2545945644378662,0.8465975385697754,0.0836333433787028,0.00622626055547526,2.5166672790475357,3.078401826611054,3.2567981031459383,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...
4,0.12,1.2743067741394043,0.8352514622184447,0.10334555308024089,-0.00511981579585552,2.6742694620132084,3.0247461431208698,3.6922309758753795,/data/ephraim/datasets/known_noise/undiff/exp_...,Namespace(enhanced_dir='/data/ephraim/datasets...


In [47]:
list(np.logspace(-2,0.7,num=40))

[0.01,
 0.011728183955791524,
 0.013755029890088569,
 0.01613215208683696,
 0.018920084727722997,
 0.02218982341458972,
 0.02602463309528382,
 0.03052216843234688,
 0.035796960610421716,
 0.04198333390972491,
 0.04923882631706739,
 0.05774820128138352,
 0.0677281527744142,
 0.07943282347242814,
 0.09316027658125524,
 0.10926008611173785,
 0.1281422388944084,
 0.15028757502606052,
 0.17626003261754578,
 0.2067210086592391,
 0.24244620170823283,
 0.2843453653017093,
 0.33348547512351867,
 0.39111789988331647,
 0.45871026782343877,
 0.5379838403443686,
 0.6309573444801934,
 0.7399983804321428,
 0.8678837132695973,
 1.0178699841461256,
 1.1937766417144369,
 1.4000832056153938,
 1.6420433388871634,
 1.9258186341850783,
 2.2586355207213766,
 2.6489692876105297,
 3.10675990983383,
 3.643665172900946,
 4.27335754210932,
 5.011872336272722]

In [26]:
p_path = "/data/ephraim/datasets/known_noise/undiff/exp_ar_h/c/snr10_50_0.1.pickle"
with open(p_path, "rb") as handle:
    loss_array_snr10_50_01 = pd.read_pickle(handle)
p_path = "/data/ephraim/datasets/known_noise/undiff/exp_ar_h/c/snr10_50_0.11.pickle"
with open(p_path, "rb") as handle:
    loss_array_snr10_50_11 = pd.read_pickle(handle)
p_path = "/data/ephraim/datasets/known_noise/undiff/exp_ar_h/c/snr5_50_0.11.pickle"
with open(p_path, "rb") as handle:
    loss_array_snr5_50_11 = pd.read_pickle(handle)

In [27]:
loss_array_snr5_50_11

[tensor(92648.7734, device='cuda:0', requires_grad=True)]

In [16]:
loss_array_snr10_50_01

[tensor(89082.4688, device='cuda:0', requires_grad=True)]

In [18]:
loss_array_snr10_50_11

[tensor(89558.6953, device='cuda:0', requires_grad=True)]